In [8]:
import numpy
# scipy.special für die Sigmoid-Funktion expit()
import scipy.special
# Bibliothek zum Plotten von Arrays
import matplotlib.pyplot as plt
# sicherstellen, dass die Plots in diesem Notebook und nicht in einem externen Fenster angezeigt werden
%matplotlib inline

# Definition der neuronalen Netzwerkklasse
class neuralNetwork:

    # Initialisierung des neuronalen Netzwerks
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # Anzahl der Nodes in jeder Input-, hidden und output layer festlegen
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # Gewichtsmatrizen, wih und who
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # Lernrate
        self.lr = learningrate

        # Aktivierungsfunktion ist die Sigmoid-Funktion
        self.activation_function = lambda x: scipy.special.expit(x)

        pass

    # Trainieren des neuronalen Netzwerks
    def train(self, inputs_list, targets_list):
        # Inputs in ein 2D-Array umwandeln
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T

        # Signale in dem Hidden Layer berechnen
        hidden_inputs = numpy.dot(self.wih, inputs)
        # Signale berechnen, die aus dem Hidden Layer hervorgehen
        hidden_outputs = self.activation_function(hidden_inputs)

        # Signale in dem endgültigen Output Layer berechnen
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # Signale berechnen, die aus dem endgültigen Output Layer  hervorgehen
        final_outputs = self.activation_function(final_inputs)

        # Fehler in dem Output Layer ist (Ziel - Ist)
        output_errors = targets - final_outputs
        # Fehler in dem Hidden Layer ist der output_errors, aufgeteilt nach Gewichten, neu kombiniert an den Hidden Nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)

        # Gewichte für die Verbindungen zwischen den hidden und den Output Layers aktualisieren
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)),
                                        numpy.transpose(hidden_outputs))

        # Gewichte für die Verbindungen zwischen den Input- und den Hidden Layern aktualisieren
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)),
                                        numpy.transpose(inputs))

        pass

    # Abfragen des neuronalen Netzwerks
    def query(self, inputs_list):
        # Input List in ein 2D-Array umwandeln
        inputs = numpy.array(inputs_list, ndmin=2).T

        # Signale in die Hidden Layern berechnen
        hidden_inputs = numpy.dot(self.wih, inputs)
        # Signale berechnen, die aus dem Hidden Layer hervorgehen
        hidden_outputs = self.activation_function(hidden_inputs)

        # Signale in den endgültigen Output Layer berechnen
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # Signale berechnen, die aus dem endgültigen Output Layer hervorgehen
        final_outputs = self.activation_function(final_inputs)

        return final_outputs


# Anzahl an input, hidden und output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 1

# Lernrate
learning_rate = 0.05

# Erstelle Instanz des neuronalen Netzwerks
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
with open("../data/fashion-mnist_train.csv", 'r') as training_data_file:
    training_data_list = training_data_file.readlines()[1:]

with open("../data/fashion-mnist_test.csv", 'r') as test_data_file:
    test_data_list = test_data_file.readlines()[1:]
# Erstellen Sie eine Liste von 10 Modellen
models = []
for i in range(10):
    models.append(neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate))
# Epochen ist die Anzahl der Male, die der Trainingsdatensatz zum Training verwendet wird
epochs = 1

# Erstellen Sie eine Liste von 10 Ensembles, die jeweils 10 Modelle enthalten
ensembles = []
for _ in range(10):
    ensemble = []
    for i in range(10):
        ensemble.append(neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate))
    ensembles.append(ensemble)


# Zusätzliche Vorbereitungen für Auswertung
training_details = {
    'class_samples': {},
    'ensemble_predictions': []
}

# Trainiere jedes Modell in jedem Ensemble auf die jeweilige Klasse gegen den Rest
for ensemble_index, ensemble in enumerate(ensembles):
    ensemble_predictions = []
    for class_index in range(10):
        # Zähle Samples pro Klasse
        class_samples = [record for record in training_data_list
                         if int(record.split(',')[0]) == class_index]
        training_details['class_samples'][class_index] = len(class_samples)

        for model in ensemble:
            class_specific_predictions = []
            for record in class_samples:
                all_values = record.split(',')
                inputs = (numpy.asarray(all_values[1:], dtype=float) / 255.0 * 0.99) + 0.01
                target = numpy.array([0.99 if int(all_values[0]) == class_index else 0.01])

                # Trainiere Modell
                model.train(inputs, target)

                # Speichere Vorhersagen für spätere Analyse
                prediction = model.query(inputs)[0]
                class_specific_predictions.append({
                    'true_class': int(all_values[0]),
                    'prediction': prediction
                })

            ensemble_predictions.extend(class_specific_predictions)

    training_details['ensemble_predictions'].append(ensemble_predictions)

# Zusätzliche Analysemöglichkeiten vorbereiten
training_details['total_ensembles'] = len(ensembles)
training_details['total_models_per_ensemble'] = len(ensembles[0])

KeyboardInterrupt: 

In [6]:
# Testen Sie jedes Bild mit allen Modellen in allen Ensembles
scorecard = []
for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (numpy.asarray(all_values[1:], dtype=float) / 255.0 * 0.99) + 0.01
    ensemble_outputs = []
    for ensemble in ensembles:
        outputs = [model.query(inputs) for model in ensemble]
        predicted_labels = [i for i, output in enumerate(outputs) if output[0] > 0.5]
        ensemble_outputs.append(predicted_labels)
    # Aggregieren Sie die Vorhersagen aller Ensembles
    all_predictions = sum(ensemble_outputs, [])
    if all_predictions:
        final_prediction = max(set(all_predictions), key=all_predictions.count)
        if correct_label == final_prediction:
            scorecard.append(1)
        else:
            scorecard.append(0)
    else:
        # Behandeln Sie den Fall, in dem kein Modell ein Label mit einer Konfidenz > 0.5 vorhersagt
        scorecard.append(0)

In [7]:
# Berechne die Leistung
performance = sum(scorecard) / len(scorecard)
print(f'Performance: {performance}')



Performance: 0.1
